In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pathlib import Path
import os
import jsonlines
import shutil

from string import punctuation
import string
import numpy as np

In [34]:
#lets create the folder structure
cwd = os.getcwd()
root='aclImdb'
p = Path(root)
if not p.exists():
    os.mkdir(root)
    os.makedirs(cwd + '/'+root+'/test/result')
    os.makedirs(cwd + '/'+root+'/test/method')
    os.makedirs(cwd + '/'+root+'/test/background')
    os.makedirs(cwd + '/'+root+'/train/result')
    os.makedirs(cwd + '/'+root+'/train/method')
    os.makedirs(cwd + '/'+root+'/train/background')
    

test_result_folder=cwd + '/'+root+'/test/result/'
test_method_folder=cwd + '/'+root+'/test/method/'
test_background_folder=cwd + '/'+root+'/test/background/'
train_result_folder=cwd + '/'+root+'/train/result/'
train_method_folder=cwd + '/'+root+'/train/method/'
train_background_folder=cwd + '/'+root+'/train/background/'

In [35]:
def create_folder(folder_name):
    p = Path(folder_name)
    if p.exists():
        shutil.rmtree(p)
        print('Deleteing old LSTM_SMOTE folder')
    try:
        print("Creating_folder")
        os.mkdir(folder_name)
    except OSError:
        print("Creation of the directory %s failed" % 'output')
folder_name='XLNET_intent_output'
create_folder(folder_name)

Creating_folder


In [36]:
def print_and_write(folder_name,text):
    file_path=folder_name+"/output.txt"
    p = Path(file_path)
    if p.exists():
        f=open(file_path, "a+")
        f.write("\n"+text)
        print(text)
        f.close()
    else:
        f= open(file_path,"w+")
        f.write("\n"+text)
        print(text)
        f.close()

In [37]:
def load_citation_intent_corpus(file_path,do_print=False,folder_name=''):

    result_list = []
    method_list = []
    background_list = []
    result_num = 0
    method_num = 0
    backgroud_num = 0

    for ex in jsonlines.open(file_path):
        text = ex.get('string')
        label = ex.get('label')

        if label == 'result':
            result_num += 1
            result_list.append(text)

        if label == 'method':
            method_num += 1
            method_list.append(text)

        if label == 'background':
            backgroud_num += 1
            background_list.append(text)

    if do_print:
        print_and_write(folder_name,"Number of examples of type :result= "+ str(result_num))
        print_and_write(folder_name,"Number of examples of type :method= "+str(method_num))
        print_and_write(folder_name,"Number of examples of type :background="+str(backgroud_num))

    return result_list,method_list,background_list

In [38]:
train_path='./scicite_data/train.jsonl'
val_path='./scicite_data/dev.jsonl'
test_path='./scicite_data/test.jsonl'
print_and_write(folder_name,"\nDetails of training folder")
train_result_list,train_method_list,train_background_list=load_citation_intent_corpus(train_path,True,folder_name)
print_and_write(folder_name,"\nDetails of validation folder")
val_result_list,val_method_list,val_background_list=load_citation_intent_corpus(val_path,True,folder_name)
print_and_write(folder_name,"\nDetails of testing folder")
test_result_list,test_method_list,test_background_list=load_citation_intent_corpus(test_path,True,folder_name)



train_result_list=train_result_list+val_result_list
train_method_list=train_method_list+val_method_list
train_background_list=train_background_list+val_background_list





Details of training folder
Number of examples of type :result= 1109
Number of examples of type :method= 2294
Number of examples of type :background=4840

Details of validation folder
Number of examples of type :result= 123
Number of examples of type :method= 255
Number of examples of type :background=538

Details of testing folder
Number of examples of type :result= 259
Number of examples of type :method= 605
Number of examples of type :background=997


In [39]:
print(len(train_result_list))
print(len(train_method_list))
print(len(train_background_list))


1232
2549
5378


In [40]:
from sklearn.model_selection import train_test_split
def write_data(list_to_write,folder):
    count=0
    for data in list_to_write:
        text_file = open(folder+str(count)+"_1.txt", "w")
        text_file.write(data)
        text_file.close()
        count+=1
    
        

In [41]:
write_data(train_result_list,train_result_folder)
write_data(train_method_list,train_method_folder)
write_data(train_background_list,train_background_folder)
write_data(test_result_list,test_result_folder)
write_data(test_method_list,test_method_folder)
write_data(test_background_list,test_background_folder)

In [42]:
SCRIPTS_DIR = 'xlnet' #@param {type:"string"}
DATA_DIR = 'aclImdb' #@param {type:"string"}
OUTPUT_DIR = 'xlnet_output' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'xlnet_cased_L-24_H-1024_A-16' #@param {type:"string"}
CHECKPOINT_DIR = 'xlnet_checkpoint' #@param {type:"string"}

In [43]:
train_command = "python run_classifier.py \
  --do_train=True \
  --do_eval=True \
  --eval_all_ckpt=True \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=8 \
  --eval_batch_size=8 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=4000 \
  --warmup_steps=500 \
  --save_steps=500 \
  --iterations=500"

! {train_command}





W1202 12:49:47.938147 140565402752832 module_wrapper.py:139] From run_classifier.py:637: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1202 12:49:47.938251 140565402752832 module_wrapper.py:139] From run_classifier.py:637: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1202 12:49:47.938348 140565402752832 module_wrapper.py:139] From run_classifier.py:661: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W1202 12:49:47.978405 140565402752832 module_wrapper.py:139] From /home/vikas/Desktop/CI_NLP/Phase1/Intent/XLNET/xlnet-master/model_utils.py:27: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W1202 12:49:47.978584 140565402752832 module_wrapper.py:139] From /home/vikas/Desktop/CI_NLP/Phase1/Intent/XLNET/xlnet-master/model_utils.py:36: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info in


W1202 12:49:51.803713 140565402752832 module_wrapper.py:139] From run_classifier.py:450: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

INFO:tensorflow:Input tfrecord file xlnet_output/spiece.model.len-128.train.tf_record
I1202 12:49:51.803958 140565402752832 run_classifier.py:459] Input tfrecord file xlnet_output/spiece.model.len-128.train.tf_record
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W1202 12:49:51.814790 140565402752832 deprecation.py:323] From /home/vikas/anaconda3/envs/albert/lib/python3.6/site-packages/tensorflow_core/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instr

INFO:tensorflow:Done calling model_fn.
I1202 12:50:05.955188 140565402752832 estimator.py:1150] Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
I1202 12:50:05.956012 140565402752832 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
I1202 12:50:09.559843 140565402752832 monitored_session.py:240] Graph was finalized.
2019-12-02 12:50:09.564968: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-12-02 12:50:09.596450: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2208000000 Hz
2019-12-02 12:50:09.597772: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56168ed8b0b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2019-12-02 12:50:09.597787: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2019-1